# This Notebook is for data processing development

Here we will calculate the model parameters

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./../data/database.csv', sep=';')

In [3]:
data.head()

,ID_PROBLEMA,ID_PACIENTE,ID PACIENTE,FECHA_NACIMIENTO_PAC,ID MÉDICO,UNIDAD_NEGOCIO,ESPECIALIDAD,ID_LUGAR,CENTRO,FECHA_EVOLUCION,...,TEXTO_PROBLEMA,TEXTO_LIBRE,CONCEPT_ID_CIE,DESCRIPTION_CIE,EDAD,ETIQUETA_GES,ID_AGRUPADOR,POTENCIAL_1,POTENCIAL_2,POTENCIAL_3
0,17648485,211402,1399,30/07/1943,1,CONSULTAS,OTORRINO,21.0,CMD PEDRO DE VALDIVIA,29/07/2019,...,NaN,NaN,H903,"HIPOACUSIA NEUROSENSORIAL, BILATERAL",76,POSIBLE,59.0,56.0,NaN,NaN
1,17644905,5845159,1399,30/07/1943,1,CONSULTAS,OTORRINO,21.0,CMD PEDRO DE VALDIVIA,29/07/2019,...,NaN,NaN,H903,"HIPOACUSIA NEUROSENSORIAL, BILATERAL",76,NO GES,77.0,56.0,NaN,NaN
2,17730364,10199783,1399,30/07/1943,1,CONSULTAS,OTORRINO,21.0,CMD PEDRO DE VALDIVIA,29/07/2019,...,NaN,NaN,H903,"HIPOACUSIA NEUROSENSORIAL, BILATERAL",76,POSIBLE,56.0,NaN,NaN,NaN
3,17677869,10824784,5268,08/09/1957,1,CONSULTAS,OTORRINO,21.0,CMD PEDRO DE VALDIVIA,02/08/2019,...,NaN,NaN,E039,"HIPOTIROIDISMO, NO ESPECIFICADO",61,NaN,NaN,76.0,NaN,NaN
4,17656698,3767945,5751,05/12/1958,1,CONSULTAS,OTORRINO,21.0,CMD PEDRO DE VALDIVIA,09/08/2019,...,NaN,NaN,H903,"HIPOACUSIA NEUROSENSORIAL, BILATERAL",60,NO GES,56.0,NaN,NaN,NaN
